In [1]:
import re

In [2]:
def remove_transform_from_path(path_text: str) -> str:
    """
    Takes an SVG <path> tag string and:
    - Removes transform="translate(tx,ty)"
    - Adds (tx, ty) to all coordinates in the `d` attribute
    Then returns the modified string.
    """
    # 1) Extract tx, ty from the transform attribute
    m_tx = re.search(
        r'transform=(["\'])\s*translate\(\s*([-+]?\d*\.?\d+)\s*,\s*([-+]?\d*\.?\d+)\s*\)\s*\1',
        path_text
    )
    if not m_tx:
        # If there's no transform, return as is
        return path_text

    tx, ty = float(m_tx.group(2)), float(m_tx.group(3))

    # 2) Extract the content of the `d` attribute
    m_d = re.search(r'd=(["\'])(?P<d>.*?)\1', path_text)
    if not m_d:
        # If `d` attribute not found, just remove transform and return
        return re.sub(r'\s+transform=(["\']).*?\1', '', path_text)

    d_orig = m_d.group('d')

    # 3) Find coordinate pairs “x,y” and shift them
    def shift_coord(m):
        x, y = float(m.group(1)), float(m.group(2))
        x2, y2 = x + tx, y + ty

        def fmt(v: float) -> str:
            return str(int(v)) if v.is_integer() else ('%f' % v).rstrip('0').rstrip('.')

        return fmt(x2) + ',' + fmt(y2)

    d_shifted = re.sub(r'([-+]?\d*\.?\d+),\s*([-+]?\d*\.?\d+)', shift_coord, d_orig)

    # 4) Replace the old `d` attribute with the new shifted one, and remove transform
    start, end = m_d.span()
    before_d = path_text[:start]
    after_d  = path_text[end:]
    new_path = before_d + f'd="{d_shifted}"' + after_d
    new_path = re.sub(r'\s+transform=(["\']).*?\1', '', new_path)

    return new_path


In [3]:
def extract_paths(svg):
    """
    Extracts all <path> opening tags from an SVG string,
    removes transform attributes, and returns a list
    of cleaned path tags in their original order.
    """
    # ① Extract only <path> tags in order
    tag_pattern = re.compile(r'<path\b[^>]*>', flags=re.IGNORECASE)
    raw_tags = [m.group(0) for m in tag_pattern.finditer(svg)]
    
    # ② Clean each path tag string
    return [remove_transform_from_path(t) for t in raw_tags]


In [4]:
def optimize_svg_path_vt(elem: str) -> str:
    """
    Takes a <path> element in M x,y L ... Z format,
    and returns the shortest equivalent path code that
    draws the same shape.
    Optimizes all "M…Z" subpaths in order.
    """
    # --- Extract attributes --------------------------------------------------
    d_match = re.search(r'd="([^"]+)"', elem)
    if not d_match:
        return elem
    d_raw = d_match.group(1)

    fill_m = re.search(r'fill="([^"]+)"', elem)
    fill = fill_m.group(1) if fill_m else None

    # --- Split by subpaths (M…Z blocks) -------------------------------------
    subpaths = re.findall(r'M[^M]*?Z', d_raw)

    optimized_subs = []
    for sub in subpaths:
        # --- Convert string to coordinate list -------------------------------
        tokens = re.findall(r'[MLZ]|-?\d+', sub)
        pts, cmd, i = [], None, 0
        while i < len(tokens):
            t = tokens[i]
            if t in ('M', 'L', 'Z'):
                cmd, i = t, i + 1
                continue
            if cmd in ('M', 'L'):
                pts.append((int(t), int(tokens[i + 1])))
                i += 2
            else:
                i += 1

        # Skip this subpath if no valid coordinates
        if not pts:
            continue

        # --- Build the shortest path command sequence ------------------------
        d_parts = [f'M{pts[0][0]} {pts[0][1]}']
        prev_x, prev_y = pts[0]

        for x, y in pts[1:]:
            dx, dy = x - prev_x, y - prev_y

            cands = []
            # ― Absolute commands ―
            if dy == 0:
                cands.append(f'H{x}')
            if dx == 0:
                cands.append(f'V{y}')
            cands.append(f'L{x} {y}')
            # ― Relative commands ―
            if dy == 0:
                cands.append(f'h{dx}')
            if dx == 0:
                cands.append(f'v{dy}')
            cands.append(f'l{dx} {dy}')

            # Choose the shortest command
            d_parts.append(min(cands, key=len))
            prev_x, prev_y = x, y

        d_parts.append('Z')
        # Skip if only "M" and "Z"
        if len(d_parts) > 2:
            optimized_subs.append(''.join(d_parts))

    if not optimized_subs:
        return None

    d_optimized = ''.join(optimized_subs)
    return f'<path d="{d_optimized}"' + (f' fill="{fill}"' if fill else '') + '/>'


In [5]:
def extract_svg_size_with_regex(svg):
    """Extract width and height from an SVG tag using regular expressions"""
    w_match = re.search(r'<svg[^>]*\bwidth=["\']([^"\']+)["\']', svg)
    h_match = re.search(r'<svg[^>]*\bheight=["\']([^"\']+)["\']', svg)
    width  = w_match.group(1) if w_match else None
    height = h_match.group(1) if h_match else None
    return width, height


In [15]:
def svg_compress(svg):
    """
    Compresses an SVG string by:
    - Extracting all paths
    - Optimizing each path to its shortest form
    - Rebuilding a compact SVG with consistent header and footer
    """
    path_list = extract_paths(svg)
    print(path_list[1])
    path_list = [optimize_svg_path_vt(p) for p in path_list]
    print(path_list[1])
    path_list = [p for p in path_list if p is not None]

    width, height = extract_svg_size_with_regex(svg)
    header = f'<svg width="384" height="384" viewBox="0 0 {width} {height}">'
    fill_background = re.search(r'fill="([^"]+)"', path_list[0])
    header += f'<rect width="{width}" height="{height}" fill="{fill_background.group(1)}"/>'
    
    footer = f'<path d="M167 200h20l-20 30h20" fill="none" stroke="white" stroke-width="{4*int(width)/256}" transform="scale({int(width)/256} {int(height)/256})" stroke-linecap="round" stroke-linejoin="round"/><path d="M167 200h20l-20 30h20" fill="none" stroke="#EDCE6A" stroke-width="{2*int(width)/256}" transform="scale({int(width)/256} {int(height)/256})" stroke-linecap="round" stroke-linejoin="round"/></svg>'
    return header + ''.join(path_list[1:]) + footer


In [16]:
with open("/home/anhndt/pysvgenius/data/test/test_svg.svg", 'r') as f:
    svg = f.read()

In [19]:
compressed_svg = svg_compress(svg)

<path d="M209,53 L215,53 L217,57 L223,59 L223,80 L220,80 L220,82 L222,82 L222,97 L211,97 L211,91 L210,90 L210,110 L211,111 L223,112 L223,129 L220,161 L219,187 L217,190 L212,189 L202,183 L196,183 L197,188 L198,194 L197,196 L193,195 L184,191 L182,189 L175,189 L169,190 L167,187 L162,185 L161,184 L161,178 L162,172 L158,168 L151,169 L147,174 L145,186 L142,186 L141,179 L137,174 L128,170 L124,166 L122,165 L110,165 L105,169 L105,178 L95,177 L86,177 L80,182 L75,182 L73,177 L69,175 L61,175 L54,178 L49,180 L48,172 L43,168 L40,167 L30,167 L23,172 L19,183 L14,182 L13,175 L8,171 L0,171 L0,103 L2,103 L2,105 L11,105 L15,109 L20,105 L27,103 L36,98 L38,96 L43,94 L47,91 L56,91 L62,94 L65,99 L72,102 L74,107 L84,106 L88,106 L93,104 L109,104 L111,99 L115,97 L115,99 L121,98 L118,95 L94,95 L87,93 L88,87 L84,85 L75,84 L74,79 L76,77 L83,77 L92,81 L97,80 L105,80 L112,85 L115,85 L117,81 L123,77 L132,71 L147,65 L155,65 L159,70 L160,72 L165,72 L168,67 L169,60 L188,55 L209,55 Z " fill="#FBF9F7" />
<path d="M209 53h6

In [22]:
path = """
<path d="M209,53 L215,53 L217,57 L223,59 L223,80 L220,80 L220,82 L222,82 L222,97 L211,97 L211,91 L210,90 L210,110 L211,111 L223,112 L223,129 L220,161 L219,187 L217,190 L212,189 L202,183 L196,183 L197,188 L198,194 L197,196 L193,195 L184,191 L182,189 L175,189 L169,190 L167,187 L162,185 L161,184 L161,178 L162,172 L158,168 L151,169 L147,174 L145,186 L142,186 L141,179 L137,174 L128,170 L124,166 L122,165 L110,165 L105,169 L105,178 L95,177 L86,177 L80,182 L75,182 L73,177 L69,175 L61,175 L54,178 L49,180 L48,172 L43,168 L40,167 L30,167 L23,172 L19,183 L14,182 L13,175 L8,171 L0,171 L0,103 L2,103 L2,105 L11,105 L15,109 L20,105 L27,103 L36,98 L38,96 L43,94 L47,91 L56,91 L62,94 L65,99 L72,102 L74,107 L84,106 L88,106 L93,104 L109,104 L111,99 L115,97 L115,99 L121,98 L118,95 L94,95 L87,93 L88,87 L84,85 L75,84 L74,79 L76,77 L83,77 L92,81 L97,80 L105,80 L112,85 L115,85 L117,81 L123,77 L132,71 L147,65 L155,65 L159,70 L160,72 L165,72 L168,67 L169,60 L188,55 L209,55 Z " fill="#FBF9F7" />
"""

In [25]:
optimize_svg_path_vt(path)

'<path d="M209 53h6l2 4l6 2V80h-3v2h2V97H211V91l-1 -1v20l1 1l12 1v17l-3 32l-1 26l-2 3l-5 -1l-10 -6h-6l1 5l1 6l-1 2l-4 -1l-9 -4l-2 -2h-7l-6 1l-2 -3l-5 -2l-1 -1v-6l1 -6l-4 -4l-7 1l-4 5l-2 12h-3l-1 -7l-4 -5l-9 -4l-4 -4l-2 -1H110l-5 4v9L95 177H86l-6 5H75l-2 -5l-4 -2H61l-7 3l-5 2l-1 -8l-5 -4l-3 -1H30l-7 5l-4 11l-5 -1l-1 -7L8 171H0V103H2v2h9l4 4l5 -4l7 -2l9 -5l2 -2l5 -2l4 -3h9l6 3l3 5l7 3l2 5l10 -1h4l5 -2h16l2 -5l4 -2v2l6 -1l-3 -3H94L87 93l1 -6L84 85L75 84L74 79l2 -2h7l9 4l5 -1h8l7 5h3l2 -4l6 -4l9 -6l15 -6h8l4 5l1 2h5l3 -5l1 -7l19 -5h21Z" fill="#FBF9F7"/>'